# 🦜🔗 LangChain agent using plan and solve

In [ ]:
!pip install -r requirements.txt

In [12]:
import os
import faiss
from collections import deque
from typing import Dict, List, Optional, Any
from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain_experimental.autonomous_agents import BabyAGI
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import SerpAPIWrapper

from dotenv import load_dotenv

load_dotenv()


True

In [24]:
embeddings = OpenAIEmbeddings()

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings.embed_query, index, InMemoryDocstore({}), {})

In [25]:
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. You have a clear focus, so you write precise todos, but not more than 4. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]

prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [26]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
max_iterations: Optional[int] = 5
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=False,
    max_iterations=max_iterations,
)

baby_agi({"objective": "Write an euphoric blog article about the code.talks conference in hamburg."})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: What tasks do I need to complete to make a todo list?
Action: TODO
Action Input: Write an euphoric blog article about the code.talks conference in hamburg.
Observation: 

1. Research the code.talks conference in Hamburg
2. Brainstorm ideas for the blog article
3. Draft the blog article
4. Edit and refine the blog article
Thought: I now know the final answer
Final Answer: A todo list for writing an euphoric blog article about the code.talks conference in Hamburg is:

1. Research the code.talks conference in Hamburg
2. Brainstorm ideas for the blog article
3. Draft the blog article
4. Edit and refine the blog article

> Finished chain.

*****TASK RESULT*****

A todo list for writing an euphoric blog article about the code.talks conference in Hamburg is:

1. Research the code.talks conference in Hamburg
2. Brainstorm ideas for the blog article
3. Draft the b

{'objective': 'Write an euphoric blog article about the code.talks conference in hamburg.'}